In [1]:
!pip install git+https://github.com/andreinechaev/nvcc4jupyter.git

  Cloning https://github.com/andreinechaev/nvcc4jupyter.git to /tmp/pip-req-build-2mtfrpln
  Running command git clone --filter=blob:none --quiet https://github.com/andreinechaev/nvcc4jupyter.git /tmp/pip-req-build-2mtfrpln
  Resolved https://github.com/andreinechaev/nvcc4jupyter.git to commit 0d2ab99cccbbc682722e708515fe9c4cfc50185a
  Preparing metadata (setup.py) ... done
  Created wheel for NVCCPlugin: filename=NVCCPlugin-0.0.2-py3-none-any.whl size=4716 sha256=75f6b0132ec5a9ef8eb665489580b9bfd50ed7141c78cb997eda6404e7fd5268
  Stored in directory: /tmp/pip-ephem-wheel-cache-14jm1pfg/wheels/a8/b9/18/23f8ef71ceb0f63297dd1903aedd067e6243a68ea756d6feea
Successfully built NVCCPlugin


In [2]:
%load_ext nvcc_plugin

created output directory at /content/src
Out bin /content/result.out


In [5]:
%%cuda --name convolution_with_shm.cu


#include <iostream>
#include <string>
#include <cuda_runtime.h>
#include <cuda.h>

#define STB_IMAGE_IMPLEMENTATION
#include "stb_image.h"

#define STB_IMAGE_WRITE_IMPLEMENTATION
#include "stb_image_write.h"

#define gpuErrchk(ans) { gpuAssert((ans), __FILE__, __LINE__); }
inline void gpuAssert(cudaError_t code, const char *file, int line, bool abort=true)
{
   if (code != cudaSuccess)
   {
      fprintf(stderr,"GPUassert: %s %s %d\n", cudaGetErrorString(code), file, line);
      if (abort) exit(code);
   }
}

bool TEST_MODE = false;

// dummy kernel that does nothing
__global__ void warmupKernel(int *dummy) {
    int i = blockIdx.x * blockDim.x + threadIdx.x;
    dummy[i] = 0;
}

// __constant__ float constantKernel[9];

__global__ void convolutionKernel(const float *input, float *output, int width,
                                    int height, const float *kernel, int kernelSize) {
    int center = (kernelSize - 1) / 2;

    // x is the column index, y is the row index
    int x = blockIdx.x * blockDim.x + threadIdx.x;
    int y = blockIdx.y * blockDim.y + threadIdx.y;

    // fill kernel into shared memory
    // if (threadIdx.x == 0 && threadIdx.y == 0) {
    //     for (int i = 0; i < kernelSize * kernelSize; ++i) {
    //         sharedKernel[i] = kernel[i];
    //     }
    // }

    extern __shared__ float sharedKernel[];
    int K2 = kernelSize * kernelSize;
    if (threadIdx.x < K2) {
        sharedKernel[threadIdx.x] = kernel[threadIdx.x];
    }

    __syncthreads();

    if (x >= center && x < width - center && y >= center && y < height - center) {
        float sum = 0.0;

        // apply convolution
        for (int ky = 0; ky < kernelSize; ++ky) {
            for (int kx = 0; kx < kernelSize; ++kx) {
                int imageX = x + kx - center;
                int imageY = y + ky - center;

                sum += input[imageY * width + imageX] * sharedKernel[ky * kernelSize + kx];
            }
        }
        // set the output pixel value
        output[y * width + x] = sum;
    }
}

void applyLoGFilterCUDA(const float *input, float *output, int width, int height, int blockDimX,
                        int blockDimY, int gridDimX, int gridDimY, int gridDimZ, int dim, float& milliseconds) {
    // define the Laplacian of Gaussian (LoG) kernel
    float kernel[9] = {
        0, 1, 0,
        1, -4, 1,
        0, 1, 0
    };
    int kernelSize = 3;

    // define 5x5 kernel
    // float kernel[25] = {
    //     0, 0, 1, 0, 0,
    //     0, 1, 2, 1, 0,
    //     1, 2, -16, 2, 1,
    //     0, 1, 2, 1, 0,
    //     0, 0, 1, 0, 0
    // };
    // int kernelSize = 5;

    int threadsPerBlock = blockDimX * blockDimY;
    if (threadsPerBlock > 1024) {
        std::cerr << "Error: The number of threads per block must be less than or equal to 1024." << std::endl;
        milliseconds = -1.0f;
        return;
    }
    // Allocate device memory
    float *d_input, *d_output, *d_kernel;
    gpuErrchk(cudaMalloc((void**)&d_input, width * height * sizeof(float)));
    gpuErrchk(cudaMalloc((void**)&d_output, width * height * sizeof(float)));
    gpuErrchk(cudaMalloc((void**)&d_kernel, kernelSize * kernelSize * sizeof(float)));

    // Copy data from host to device
    gpuErrchk(cudaMemcpy(d_input, input, width * height * sizeof(float), cudaMemcpyHostToDevice));
    gpuErrchk(cudaMemcpy(d_kernel, kernel, kernelSize * kernelSize * sizeof(float), cudaMemcpyHostToDevice));

    // Configure and launch the CUDA kernel with shared memory
    dim3 blockDim(blockDimX, blockDimY);
    dim3 gridDim(gridDimX, gridDimY, gridDimZ);
    int sharedMemorySize = kernelSize * kernelSize * sizeof(float);

    // copy kernel to constant memory
    // cudaMemcpyToSymbol(constantKernel, kernel, 9 * sizeof(float));

    // warmup kernel
    int *dummy;
    cudaMalloc((void**)&dummy, width * height * sizeof(int));
    warmupKernel<<<gridDim, blockDim>>>(dummy);
    cudaFree(dummy);

    // create cuda event for timing
    cudaEvent_t start, stop;
    cudaEventCreate(&start);
    cudaEventCreate(&stop);

    // print number of thread in each block
    std::cout << "blockDimX: " << blockDimX << std::endl;
    std::cout << "blockDimY: " << blockDimY << std::endl;
    std::cout << "gridDimX: " << gridDimX << std::endl;
    std::cout << "gridDimY: " << gridDimY << std::endl;
    std::cout << "gridDimZ: " << gridDimZ << std::endl;
    std::cout << "number of threads in each block: " << blockDimX * blockDimY << std::endl;
    std::cout << "total number of blocks: " << gridDimX * gridDimY * gridDimZ << std::endl;
    std::cout << "total number of threads: " << gridDimX * gridDimY * gridDimZ * blockDimX * blockDimY << std::endl;

    // start timing
    cudaEventRecord(start);
    convolutionKernel<<<gridDim, blockDim, sharedMemorySize>>>(d_input, d_output, width, height, d_kernel, kernelSize);
    gpuErrchk(cudaPeekAtLastError());
    gpuErrchk(cudaDeviceSynchronize());

    // stop timing
    cudaEventRecord(stop);
    cudaEventSynchronize(stop);
    milliseconds = 0;
    cudaEventElapsedTime(&milliseconds, start, stop);
    std::cout << "time taken: " << milliseconds << " ms" << std::endl;

    // copy the result back to the host
    cudaMemcpy(output, d_output, width * height * sizeof(float), cudaMemcpyDeviceToHost);

    // free device memory
    cudaFree(d_input);
    cudaFree(d_output);
    cudaFree(d_kernel);

    // destroy CUDA event
    cudaEventDestroy(start);
    cudaEventDestroy(stop);
}

void configure_grid_2D(int width, int height, const int& blockDimX, const int& blockDimY, int& gridDimX, int& gridDimY, int& gridDimZ) {
    gridDimX = (width + blockDimX - 1) / blockDimX;
    gridDimY = (height + blockDimY - 1) / blockDimY;
    gridDimZ = 1;
}

void saveOutputImagePNG(const float *outputImage, int width, int height, const std::string& filename) {
    // create output image buffer
    unsigned char *outputImageChar = new unsigned char[width * height];

    // convert output to unsigned char
    for (int i = 0; i < width * height; ++i) {
        outputImageChar[i] = (unsigned char) std::round(outputImage[i] * 255.0f);
    }

    // Save the output image
    stbi_write_png(filename.c_str(), width, height, 1, outputImageChar, 0);

    delete[] outputImageChar;
}

int main(int argc, char *argv[]) {

    // ./program <input_image> <output_image> | ./program <input_image> <output_image> <-test>
    if (argc != 3 && argc != 4) {
        std::cerr << "Usage: " << argv[0] << " input_image output_image" << std::endl;
        return 1;
    }
    else if (argc == 4 && std::string(argv[3]) != "-test") {
        std::cerr << "Usage: " << argv[0] << " input_image output_image" << std::endl;
        return 1;
    }

    if (argc == 4) {
        std::cout << "Running in test mode." << std::endl;
        TEST_MODE = true;
    }

    // Load input image
    int width, height, channels;
    unsigned char *inputImage = stbi_load(argv[1], &width, &height, &channels, 1);
    float *outputImageFloat = new float[width * height];

    if (!inputImage) {
        std::cerr << "Error loading image: " << stbi_failure_reason() << std::endl;
        return 1;
    }

    // print width and height
    std::cout << "width: " << width << std::endl;
    std::cout << "height: " << height << std::endl;

    // convert input to float
    float *inputImageFloat = new float[width * height];
    for (int i = 0; i < width * height; ++i) {
        inputImageFloat[i] = inputImage[i] / 255.0f;
    }

    if (TEST_MODE) {
        int blockDimX;
        int blockDimY;
        int gridDimX, gridDimY, gridDimZ;
        float milliseconds = 0.0f;

        int fixedDimensionsX[9] = {32, 64, 128, 256, 512, 1024, 8, 16, 32};
        int fixedDimensionsY[9] = {1, 1, 1, 1, 1, 1, 8, 16, 32};
        int numFixedDimensions = 9;

        // block combination array ex: 1 row as 1 block, 2 rows as 1 block, etc.
        int blockDimXArray[6] = {width, width / 2, width / 4, width/2, width/4, 1};
        int blockDimYArray[6] = {1, 1, 1, 2, 4, height};
        int numBlockCombinations = 6;

        // test fixed dimensions
        for (int i = 0; i < numFixedDimensions; ++i) {
            blockDimX = fixedDimensionsX[i];
            blockDimY = fixedDimensionsY[i];
            std::cout << std::endl << std::endl;

            configure_grid_2D(width, height, blockDimX, blockDimY, gridDimX, gridDimY, gridDimZ);
            std::string filename = "output_2D_" + std::to_string(blockDimX) + "_" + std::to_string(blockDimY) + ".png";
            applyLoGFilterCUDA(inputImageFloat, outputImageFloat, width, height, blockDimX, blockDimY, gridDimX, gridDimY, gridDimZ, 2, milliseconds);
            saveOutputImagePNG(outputImageFloat, width, height, filename);
        }

        // test configurable dimensions
        for (int i = 0; i < numBlockCombinations; ++i) {
            blockDimX = blockDimXArray[i];
            blockDimY = blockDimYArray[i];
            std::cout << std::endl << std::endl;

            configure_grid_2D(width, height, blockDimX, blockDimY, gridDimX, gridDimY, gridDimZ);

            applyLoGFilterCUDA(inputImageFloat, outputImageFloat, width, height, blockDimX, blockDimY, gridDimX, gridDimY, gridDimZ, 2, milliseconds);
            std::string filename = "output_2D_" + std::to_string(blockDimX) + "_" + std::to_string(blockDimY) + ".png";
            saveOutputImagePNG(outputImageFloat, width, height, filename);
        }
        return 0;
    }

    // apply Laplacian of Gaussian (LoG) filter using CUDA with shared memory

    // one block per row
    int blockDimX = width;
    int blockDimY = 1;
    int gridDimX, gridDimY, gridDimZ;

    configure_grid_2D(width, height, blockDimX, blockDimY, gridDimX, gridDimY, gridDimZ);

    float sum = 0.0f;
    float milliseconds = 0.0f;
    int count = 0;
    for (int i = 0; i < 9; ++i) {
        applyLoGFilterCUDA(inputImageFloat, outputImageFloat, width, height, blockDimX, blockDimY, gridDimX, gridDimY, gridDimZ, 2, milliseconds);
        if (milliseconds != -1.0f) {
            ++count;
            sum += milliseconds;
        }
    }

    std::cout << "average time taken: " << sum / count << " ms" << std::endl;

    saveOutputImagePNG(outputImageFloat, width, height, argv[2]);

    // clean up
    stbi_image_free(inputImage);
    delete[] outputImageFloat;

    std::cout << "Laplacian of Gaussian (LoG) edge detection completed successfully." << std::endl;

    return 0;
}


'File written in /content/src/convolution_with_shm.cu'

In [6]:
!nvcc -arch=sm_75 -o "/content/src/convolution_with_shm.o" /content/src/convolution_with_shm.cu

/content/src/stb_image.h(4275): warning #550-D: variable "old_limit" was set but never used
     unsigned int cur, limit, old_limit;
                              ^

Remark: The warnings can be suppressed with "-diag-suppress <warning-number>"

/content/src/stb_image.h(5182): warning #550-D: variable "idata_limit_old" was set but never used
                 stbi__uint32 idata_limit_old = idata_limit;
                              ^

/content/src/stb_image.h(6969): warning #550-D: variable "out_size" was set but never used
        int out_size = 0;
            ^

/content/src/stb_image.h(6970): warning #550-D: variable "delays_size" was set but never used
        int delays_size = 0;
            ^



In [7]:
!/content/src/convolution_with_shm.o /content/src/image_03.png /content/src/conv_03.png

width: 933
height: 882
blockDimX: 933
blockDimY: 1
gridDimX: 1
gridDimY: 882
gridDimZ: 1
number of threads in each block: 933
total number of blocks: 882
total number of threads: 822906
time taken: 0.135456 ms
blockDimX: 933
blockDimY: 1
gridDimX: 1
gridDimY: 882
gridDimZ: 1
number of threads in each block: 933
total number of blocks: 882
total number of threads: 822906
time taken: 0.104896 ms
blockDimX: 933
blockDimY: 1
gridDimX: 1
gridDimY: 882
gridDimZ: 1
number of threads in each block: 933
total number of blocks: 882
total number of threads: 822906
time taken: 0.10624 ms
blockDimX: 933
blockDimY: 1
gridDimX: 1
gridDimY: 882
gridDimZ: 1
number of threads in each block: 933
total number of blocks: 882
total number of threads: 822906
time taken: 0.104992 ms
blockDimX: 933
blockDimY: 1
gridDimX: 1
gridDimY: 882
gridDimZ: 1
number of threads in each block: 933
total number of blocks: 882
total number of threads: 822906
time taken: 0.10448 ms
blockDimX: 933
blockDimY: 1
gridDimX: 1
grid

In [8]:
!/content/src/convolution_with_shm.o /content/src/image_03.png /content/src/conv_03.png -test

Running in test mode.
width: 933
height: 882


blockDimX: 32
blockDimY: 1
gridDimX: 30
gridDimY: 882
gridDimZ: 1
number of threads in each block: 32
total number of blocks: 26460
total number of threads: 846720
time taken: 0.164288 ms


blockDimX: 64
blockDimY: 1
gridDimX: 15
gridDimY: 882
gridDimZ: 1
number of threads in each block: 64
total number of blocks: 13230
total number of threads: 846720
time taken: 0.09632 ms


blockDimX: 128
blockDimY: 1
gridDimX: 8
gridDimY: 882
gridDimZ: 1
number of threads in each block: 128
total number of blocks: 7056
total number of threads: 903168
time taken: 0.097952 ms


blockDimX: 256
blockDimY: 1
gridDimX: 4
gridDimY: 882
gridDimZ: 1
number of threads in each block: 256
total number of blocks: 3528
total number of threads: 903168
time taken: 0.099232 ms


blockDimX: 512
blockDimY: 1
gridDimX: 2
gridDimY: 882
gridDimZ: 1
number of threads in each block: 512
total number of blocks: 1764
total number of threads: 903168
time taken: 0.1024 ms


blockD

In [9]:
%%cuda --name convolution_without_shm.cu


#include <iostream>
#include <string>
#include <cuda_runtime.h>
#include <cuda.h>

#define STB_IMAGE_IMPLEMENTATION
#include "stb_image.h"

#define STB_IMAGE_WRITE_IMPLEMENTATION
#include "stb_image_write.h"

#define gpuErrchk(ans) { gpuAssert((ans), __FILE__, __LINE__); }
inline void gpuAssert(cudaError_t code, const char *file, int line, bool abort=true)
{
   if (code != cudaSuccess)
   {
      fprintf(stderr,"GPUassert: %s %s %d\n", cudaGetErrorString(code), file, line);
      if (abort) exit(code);
   }
}

bool TEST_MODE = false;

// dummy kernel that does nothing
__global__ void warmupKernel(int *dummy) {
    int i = blockIdx.x * blockDim.x + threadIdx.x;
    dummy[i] = 0;
}

__global__ void convolutionKernel(const float *input, float *output, int width,
                                    int height, const float *kernel, int kernelSize) {
    int center = (kernelSize - 1) / 2;

    // x is the column index, y is the row index
    int x = blockIdx.x * blockDim.x + threadIdx.x;
    int y = blockIdx.y * blockDim.y + threadIdx.y;

    // each thread writes which pixel it is processing
    // printf("x: %d, y: %d\n", x, y);

    if (x >= center && x < width - center && y >= center && y < height - center) {
        float sum = 0.0;

        // apply convolution using shared memory for the kernel
        for (int ky = 0; ky < kernelSize; ++ky) {
            for (int kx = 0; kx < kernelSize; ++kx) {
                int imageX = x + kx - center;
                int imageY = y + ky - center;

                sum += input[imageY * width + imageX] * kernel[ky * kernelSize + kx];
            }
        }

        // set the output pixel value
        output[y * width + x] = sum;
    }
}

// function to apply Laplacian of Gaussian (LoG) filter using CUDA with shared memory
void applyLoGFilterCUDA(const float *input, float *output, int width, int height, int blockDimX, int blockDimY,
                            int gridDimX, int gridDimY, int gridDimZ, int dim, float& milliseconds) {
    // define the Laplacian of Gaussian (LoG) kernel
    float kernel[9] = {
        0, 1, 0,
        1, -4, 1,
        0, 1, 0
    };
    int kernelSize = 3;

    // // define 5x5 kernel
    // float kernel[25] = {
    //     0, 0, 1, 0, 0,
    //     0, 1, 2, 1, 0,
    //     1, 2, -16, 2, 1,
    //     0, 1, 2, 1, 0,
    //     0, 0, 1, 0, 0
    // };
    // int kernelSize = 5;

    int threadsPerBlock = blockDimX * blockDimY;
    if (threadsPerBlock > 1024) {
        std::cerr << "Error: The number of threads per block must be less than or equal to 1024." << std::endl;
        return;
    }

    // allocate device memory
    float *d_input, *d_output, *d_kernel;
    gpuErrchk(cudaMalloc((void**)&d_input, width * height * sizeof(float)));
    gpuErrchk(cudaMalloc((void**)&d_output, width * height * sizeof(float)));
    gpuErrchk(cudaMalloc((void**)&d_kernel, kernelSize * kernelSize * sizeof(float)));

    // copy data from host to device
    gpuErrchk(cudaMemcpy(d_input, input, width * height * sizeof(float), cudaMemcpyHostToDevice));
    gpuErrchk(cudaMemcpy(d_kernel, kernel, kernelSize * kernelSize * sizeof(float), cudaMemcpyHostToDevice));

    // configure and launch the CUDA kernel with shared memory
    dim3 blockDim(blockDimX, blockDimY);
    dim3 gridDim(gridDimX, gridDimY, gridDimZ);

    // warmup kernel
    int *dummy;
    cudaMalloc((void**)&dummy, width * height * sizeof(int));
    warmupKernel<<<gridDim, blockDim>>>(dummy);
    cudaFree(dummy);

    // create cuda event for timing
    cudaEvent_t start, stop;
    cudaEventCreate(&start);
    cudaEventCreate(&stop);

    // print number of thread in each block
    std::cout << "blockDimX: " << blockDimX << std::endl;
    std::cout << "blockDimY: " << blockDimY << std::endl;
    std::cout << "gridDimX: " << gridDimX << std::endl;
    std::cout << "gridDimY: " << gridDimY << std::endl;
    std::cout << "gridDimZ: " << gridDimZ << std::endl;
    std::cout << "number of threads in each block: " << blockDimX * blockDimY << std::endl;
    std::cout << "total number of blocks: " << gridDimX * gridDimY * gridDimZ << std::endl;
    std::cout << "total number of threads: " << gridDimX * gridDimY * gridDimZ * blockDimX * blockDimY << std::endl;

    // start timing
    cudaEventRecord(start);
    convolutionKernel<<<gridDim, blockDim>>>(d_input, d_output, width, height, d_kernel, kernelSize);
    gpuErrchk(cudaPeekAtLastError());
    gpuErrchk(cudaDeviceSynchronize());

    // stop timing
    cudaEventRecord(stop);
    cudaEventSynchronize(stop);
    milliseconds = 0.0f;
    cudaEventElapsedTime(&milliseconds, start, stop);
    std::cout << "time taken: " << milliseconds << " ms" << std::endl;

    // copy the result back to the host
    cudaMemcpy(output, d_output, width * height * sizeof(float), cudaMemcpyDeviceToHost);

    // free device memory
    cudaFree(d_input);
    cudaFree(d_output);
    cudaFree(d_kernel);
}

void configure_grid_2D(int width, int height, const int& blockDimX, const int& blockDimY, int& gridDimX, int& gridDimY, int& gridDimZ) {
    gridDimX = (width + blockDimX - 1) / blockDimX;
    gridDimY = (height + blockDimY - 1) / blockDimY;
    gridDimZ = 1;
}

void saveOutputImagePNG(const float *outputImage, int width, int height, const std::string& filename) {
    // create output image buffer
    unsigned char *outputImageChar = new unsigned char[width * height];

    // convert output to unsigned char
    for (int i = 0; i < width * height; ++i) {
        outputImageChar[i] = (unsigned char) std::round(outputImage[i] * 255.0f);
    }

    // save the output image
    stbi_write_png(filename.c_str(), width, height, 1, outputImageChar, 0);

    delete[] outputImageChar;
}

int main(int argc, char *argv[]) {

    // ./program <input_image> <output_image> | ./program <input_image> <output_image> <-test>
    if (argc != 3 && argc != 4) {
        std::cerr << "Usage: " << argv[0] << " input_image output_image" << std::endl;
        return 1;
    }
    else if (argc == 4 && std::string(argv[3]) != "-test") {
        std::cerr << "Usage: " << argv[0] << " input_image output_image" << std::endl;
        return 1;
    }

    if (argc == 4) {
        std::cout << "Running in test mode." << std::endl;
        TEST_MODE = true;
    }

    // load input image
    int width, height, channels;
    unsigned char *inputImage = stbi_load(argv[1], &width, &height, &channels, 1);
    float *outputImageFloat = new float[width * height];

    if (!inputImage) {
        std::cerr << "Error loading image: " << stbi_failure_reason() << std::endl;
        return 1;
    }

    // print width and height
    std::cout << "width: " << width << std::endl;
    std::cout << "height: " << height << std::endl;

    // convert input to float
    float *inputImageFloat = new float[width * height];
    for (int i = 0; i < width * height; ++i) {
        inputImageFloat[i] = inputImage[i] / 255.0f;
    }

  if (TEST_MODE) {
        int blockDimX;
        int blockDimY;
        int gridDimX, gridDimY, gridDimZ;
        float milliseconds = 0.0f;

        int fixedDimensionsX[9] = {32, 64, 128, 256, 512, 1024, 8, 16, 32};
        int fixedDimensionsY[9] = {1, 1, 1, 1, 1, 1, 8, 16, 32};
        int numFixedDimensions = 9;

        // block combination array ex: 1 row as 1 block, 2 rows as 1 block, etc.
        int blockDimXArray[6] = {width, width / 2, width / 4, width/2, width/4, 1};
        int blockDimYArray[6] = {1, 1, 1, 2, 4, height};
        int numBlockCombinations = 6;

        // test fixed dimensions
        for (int i = 0; i < numFixedDimensions; ++i) {
            blockDimX = fixedDimensionsX[i];
            blockDimY = fixedDimensionsY[i];
            std::cout << std::endl << std::endl;

            configure_grid_2D(width, height, blockDimX, blockDimY, gridDimX, gridDimY, gridDimZ);
            std::string filename = "output_2D_" + std::to_string(blockDimX) + "_" + std::to_string(blockDimY) + ".png";
            applyLoGFilterCUDA(inputImageFloat, outputImageFloat, width, height, blockDimX, blockDimY, gridDimX, gridDimY, gridDimZ, 2, milliseconds);
            saveOutputImagePNG(outputImageFloat, width, height, filename);
        }

        // test configurable dimensions
        for (int i = 0; i < numBlockCombinations; ++i) {
            blockDimX = blockDimXArray[i];
            blockDimY = blockDimYArray[i];
            std::cout << std::endl << std::endl;

            configure_grid_2D(width, height, blockDimX, blockDimY, gridDimX, gridDimY, gridDimZ);

            applyLoGFilterCUDA(inputImageFloat, outputImageFloat, width, height, blockDimX, blockDimY, gridDimX, gridDimY, gridDimZ, 2, milliseconds);
            std::string filename = "output_2D_" + std::to_string(blockDimX) + "_" + std::to_string(blockDimY) + ".png";
            saveOutputImagePNG(outputImageFloat, width, height, filename);
        }
        return 0;
    }


    // apply Laplacian of Gaussian (LoG) filter using CUDA with shared memory

    // one block per row
    int blockDimX = width;
    int blockDimY = 1;
    int gridDimX, gridDimY, gridDimZ;

    configure_grid_2D(width, height, blockDimX, blockDimY, gridDimX, gridDimY, gridDimZ);

    float sum = 0.0f;
    float milliseconds = 0.0f;
    int count = 0;

    for (int i = 0; i < 10; ++i) {
        applyLoGFilterCUDA(inputImageFloat, outputImageFloat, width, height, blockDimX, blockDimY, gridDimX, gridDimY, gridDimZ, 2, milliseconds);
        if (milliseconds != -1.0f) {
            ++count;
            sum += milliseconds;
        }
    }

    std::cout << "average time taken: " << sum / count << " ms" << std::endl;

    saveOutputImagePNG(outputImageFloat, width, height, argv[2]);

    // clean up
    stbi_image_free(inputImage);
    delete[] outputImageFloat;
    return 0;
}


'File written in /content/src/convolution_without_shm.cu'

In [10]:
!nvcc -arch=sm_75 -o "/content/src/convolution_without_shm.o" /content/src/convolution_without_shm.cu

/content/src/stb_image.h(4275): warning #550-D: variable "old_limit" was set but never used
     unsigned int cur, limit, old_limit;
                              ^

Remark: The warnings can be suppressed with "-diag-suppress <warning-number>"

/content/src/stb_image.h(5182): warning #550-D: variable "idata_limit_old" was set but never used
                 stbi__uint32 idata_limit_old = idata_limit;
                              ^

/content/src/stb_image.h(6969): warning #550-D: variable "out_size" was set but never used
        int out_size = 0;
            ^

/content/src/stb_image.h(6970): warning #550-D: variable "delays_size" was set but never used
        int delays_size = 0;
            ^



In [11]:
!/content/src/convolution_without_shm.o /content/src/image_03.png /content/src/conv_03_2.png

width: 933
height: 882
blockDimX: 933
blockDimY: 1
gridDimX: 1
gridDimY: 882
gridDimZ: 1
number of threads in each block: 933
total number of blocks: 882
total number of threads: 822906
time taken: 0.13072 ms
blockDimX: 933
blockDimY: 1
gridDimX: 1
gridDimY: 882
gridDimZ: 1
number of threads in each block: 933
total number of blocks: 882
total number of threads: 822906
time taken: 0.10448 ms
blockDimX: 933
blockDimY: 1
gridDimX: 1
gridDimY: 882
gridDimZ: 1
number of threads in each block: 933
total number of blocks: 882
total number of threads: 822906
time taken: 0.104608 ms
blockDimX: 933
blockDimY: 1
gridDimX: 1
gridDimY: 882
gridDimZ: 1
number of threads in each block: 933
total number of blocks: 882
total number of threads: 822906
time taken: 0.10608 ms
blockDimX: 933
blockDimY: 1
gridDimX: 1
gridDimY: 882
gridDimZ: 1
number of threads in each block: 933
total number of blocks: 882
total number of threads: 822906
time taken: 0.104384 ms
blockDimX: 933
blockDimY: 1
gridDimX: 1
gridD

In [12]:
!/content/src/convolution_without_shm.o /content/src/image_03.png /content/src/conv_03_2.png -test

Running in test mode.
width: 933
height: 882


blockDimX: 32
blockDimY: 1
gridDimX: 30
gridDimY: 882
gridDimZ: 1
number of threads in each block: 32
total number of blocks: 26460
total number of threads: 846720
time taken: 0.176704 ms


blockDimX: 64
blockDimY: 1
gridDimX: 15
gridDimY: 882
gridDimZ: 1
number of threads in each block: 64
total number of blocks: 13230
total number of threads: 846720
time taken: 0.096256 ms


blockDimX: 128
blockDimY: 1
gridDimX: 8
gridDimY: 882
gridDimZ: 1
number of threads in each block: 128
total number of blocks: 7056
total number of threads: 903168
time taken: 0.114816 ms


blockDimX: 256
blockDimY: 1
gridDimX: 4
gridDimY: 882
gridDimZ: 1
number of threads in each block: 256
total number of blocks: 3528
total number of threads: 903168
time taken: 0.100352 ms


blockDimX: 512
blockDimY: 1
gridDimX: 2
gridDimY: 882
gridDimZ: 1
number of threads in each block: 512
total number of blocks: 1764
total number of threads: 903168
time taken: 0.102496 ms


blo

In [13]:
%%cuda --name conv_shm.cu

#include <iostream>
#include <string>
#include <cuda_runtime.h>
#include <cuda.h>

#define STB_IMAGE_IMPLEMENTATION
#include "stb_image.h"

#define STB_IMAGE_WRITE_IMPLEMENTATION
#include "stb_image_write.h"

bool TEST_MODE = false;

// dummy kernel that does nothing
__global__ void warmupKernel(int *dummy) {
    int i = blockIdx.x * blockDim.x + threadIdx.x;
    dummy[i] = 0;
}

__global__ void convolutionKernelShared(const float *input, float *output, int width, int height, const float *kernel, int kernelSize) {

    int iy = blockIdx.x + (kernelSize - 1) / 2;

    int ix = threadIdx.x + (kernelSize - 1) / 2;

    // center of kernel in both dimensions
    int center = (kernelSize - 1) / 2;

    int idx = iy * width + ix;

    int threadId = threadIdx.x;
    int K2 = kernelSize * kernelSize;

    // shared memory for the kernel
    extern __shared__ float sdata[];

    // load kernel into shared memory
    if (threadId < K2) {
        sdata[threadId] = kernel[threadId];
    }

    __syncthreads();

    float sum = 0.0;

    // check if the thread is within the image
    if (idx < width * height) {
        for (int ki = 0; ki < kernelSize; ++ki) {
            for (int kj = 0; kj < kernelSize; ++kj) {
                int imageX = ix + kj - center;
                int imageY = iy + ki - center;

                sum += input[imageY * width + imageX] * sdata[ki * kernelSize + kj];
            }
        }

        output[idx] = sum;
    }
}

void applyLoGFilterCUDA(const float *input, float *output, int width, int height, float& milliseconds) {
    // define the  kernel
    float kernel[9] = {
        0, 1, 0,
        1, -4, 1,
        0, 1, 0
    };
    int kernelSize = 3;

    // define 5x5 kernel
    // float kernel[25] = {
    //     0, 0, 1, 0, 0,
    //     0, 1, 2, 1, 0,
    //     1, 2, -16, 2, 1,
    //     0, 1, 2, 1, 0,
    //     0, 0, 1, 0, 0
    // };
    // int kernelSize = 5;


    // allocate device memory
    float *d_input, *d_output, *d_kernel;
    cudaMalloc((void**)&d_input, width * height * sizeof(float));
    cudaMalloc((void**)&d_output, width * height * sizeof(float));
    cudaMalloc((void**)&d_kernel, kernelSize * kernelSize * sizeof(float));

    // copy data from host to device
    cudaMemcpy(d_input, input, width * height * sizeof(float), cudaMemcpyHostToDevice);
    cudaMemcpy(d_kernel, kernel, kernelSize * kernelSize * sizeof(float), cudaMemcpyHostToDevice);

    // configure and launch the CUDA kernel with shared memory
    int numBlocks = height - kernelSize + 1;
    int threadsPerBlock = width - kernelSize + 1;
    std::cout << "numBlocks: " << numBlocks << std::endl;
    std::cout << "threadsPerBlock: " << threadsPerBlock << std::endl;

    // warmup kernel
    int *dummy;
    cudaMalloc((void**)&dummy, numBlocks * threadsPerBlock * sizeof(int));
    warmupKernel<<<numBlocks, threadsPerBlock>>>(dummy);
    cudaFree(dummy);

    // create cuda event for timing
    cudaEvent_t start, stop;
    cudaEventCreate(&start);
    cudaEventCreate(&stop);

    // start timing
    cudaEventRecord(start);
    convolutionKernelShared<<<numBlocks, threadsPerBlock, kernelSize * kernelSize * sizeof(float)>>>(d_input, d_output, width, height, d_kernel, kernelSize);
    cudaEventRecord(stop);

    // stop timing
    cudaEventSynchronize(stop);
    milliseconds = 0;
    cudaEventElapsedTime(&milliseconds, start, stop);
    std::cout << "Time taken: " << milliseconds << " ms" << std::endl;

    // copy the result back to the host
    cudaMemcpy(output, d_output, width * height * sizeof(float), cudaMemcpyDeviceToHost);

    // free device memory
    cudaFree(d_input);
    cudaFree(d_output);
    cudaFree(d_kernel);
}

int main(int argc, char *argv[]) {

    // ./program <input_image> <output_image> | ./program <input_image> <output_image> <-test>
    if (argc != 3 && argc != 4) {
        std::cerr << "Usage: " << argv[0] << " input_image output_image" << std::endl;
        return 1;
    }

    // load input image
    int width, height, channels;
    unsigned char *inputImage = stbi_load(argv[1], &width, &height, &channels, 1);
    float *outputImageFloat = new float[width * height];

    if (!inputImage) {
        std::cerr << "Error loading image: " << stbi_failure_reason() << std::endl;
        return 1;
    }

    // print width and height
    std::cout << "width: " << width << std::endl;
    std::cout << "height: " << height << std::endl;

    // convert input to float
    float *inputImageFloat = new float[width * height];
    for (int i = 0; i < width * height; ++i) {
        inputImageFloat[i] = inputImage[i] / 255.0f;
    }

    float sum = 0.0f;
    float milliseconds = 0.0f;
    for (int i = 0; i < 10; ++i) {
        // apply Laplacian of Gaussian (LoG) filter using CUDA with shared memory
        applyLoGFilterCUDA(inputImageFloat, outputImageFloat, width, height, milliseconds);
        sum += milliseconds;
    }
    milliseconds = sum / 10.0f;
    std::cout << "Average time taken: " << milliseconds << " ms" << std::endl;

    // apply Laplacian of Gaussian (LoG) filter using CUDA with shared memory
    // applyLoGFilterCUDA(inputImageFloat, outputImageFloat, width, height, milliseconds);

    // create output image buffer
    unsigned char *outputImage = new unsigned char[width * height];

    // convert output to unsigned char
    for (int i = 0; i < width * height; ++i) {
        outputImage[i] = (unsigned char) std::round(outputImageFloat[i] * 255.0f);
    }

    // save the output image
    stbi_write_png(argv[2], width, height, 1, outputImage, 0);

    // clean up
    stbi_image_free(inputImage);
    delete[] outputImage;
    delete[] outputImageFloat;

    std::cout << "Laplacian of Gaussian (LoG) edge detection completed successfully." << std::endl;

    return 0;
}


'File written in /content/src/conv_shm.cu'

In [14]:
!nvcc -arch=sm_75 -o "/content/src/conv_shm.o" /content/src/conv_shm.cu

/content/src/stb_image.h(4275): warning #550-D: variable "old_limit" was set but never used
     unsigned int cur, limit, old_limit;
                              ^

Remark: The warnings can be suppressed with "-diag-suppress <warning-number>"

/content/src/stb_image.h(5182): warning #550-D: variable "idata_limit_old" was set but never used
                 stbi__uint32 idata_limit_old = idata_limit;
                              ^

/content/src/stb_image.h(6969): warning #550-D: variable "out_size" was set but never used
        int out_size = 0;
            ^

/content/src/stb_image.h(6970): warning #550-D: variable "delays_size" was set but never used
        int delays_size = 0;
            ^



In [15]:
!/content/src/conv_shm.o /content/src/image_03.png /content/src/conv_03_3.png

width: 933
height: 882
numBlocks: 880
threadsPerBlock: 931
Time taken: 0.110592 ms
numBlocks: 880
threadsPerBlock: 931
Time taken: 0.096288 ms
numBlocks: 880
threadsPerBlock: 931
Time taken: 0.097216 ms
numBlocks: 880
threadsPerBlock: 931
Time taken: 0.098304 ms
numBlocks: 880
threadsPerBlock: 931
Time taken: 0.0992 ms
numBlocks: 880
threadsPerBlock: 931
Time taken: 0.102144 ms
numBlocks: 880
threadsPerBlock: 931
Time taken: 0.101248 ms
numBlocks: 880
threadsPerBlock: 931
Time taken: 0.098336 ms
numBlocks: 880
threadsPerBlock: 931
Time taken: 0.098112 ms
numBlocks: 880
threadsPerBlock: 931
Time taken: 0.097856 ms
Average time taken: 0.0999296 ms
Laplacian of Gaussian (LoG) edge detection completed successfully.


In [17]:
%%cuda --name conv_without_sh.cu


#include <iostream>
#include <string>
#include <cuda_runtime.h>
#include <cuda.h>

#define STB_IMAGE_IMPLEMENTATION
#include "stb_image.h"

#define STB_IMAGE_WRITE_IMPLEMENTATION
#include "stb_image_write.h"

bool TEST_MODE = false;


// dummy kernel that does nothing
__global__ void warmupKernel(int *dummy) {
    int i = blockIdx.x * blockDim.x + threadIdx.x;
    dummy[i] = 0;
}

__global__ void convolutionKernelShared(const float *input, float *output, int width, int height, const float *kernel, int kernelSize) {

    // each block is assigned to a row of an image, iy integer index of y
    int iy = blockIdx.x + (kernelSize - 1) / 2;

    // each thread is assigned to a pixel in a row, ix integer index of x
    int ix = threadIdx.x + (kernelSize - 1) / 2;

    // center of kernel in both dimensions
    int center = (kernelSize - 1) / 2;

    int idx = iy * width + ix;

    float sum = 0.0;
    // check if the thread is within the image
    if (idx < width * height) {
        for (int ki = 0; ki < kernelSize; ++ki) {
            for (int kj = 0; kj < kernelSize; ++kj) {
                int imageX = ix + kj - center;
                int imageY = iy + ki - center;

                sum += input[imageY * width + imageX] * kernel[ki * kernelSize + kj];
            }
        }

        output[idx] = sum;
    }
}

void applyLoGFilterCUDA(const float *input, float *output, int width, int height, float& milliseconds) {
    // define the kernel
    float kernel[9] = {
        0, 1, 0,
        1, -4, 1,
        0, 1, 0
    };

    // allocate device memory
    float *d_input, *d_output, *d_kernel;
    cudaMalloc((void**)&d_input, width * height * sizeof(float));
    cudaMalloc((void**)&d_output, width * height * sizeof(float));
    cudaMalloc((void**)&d_kernel, 9 * sizeof(float));

    // copy data from host to device
    cudaMemcpy(d_input, input, width * height * sizeof(float), cudaMemcpyHostToDevice);
    cudaMemcpy(d_kernel, kernel, 9 * sizeof(float), cudaMemcpyHostToDevice);

    // configure and launch the CUDA kernel with shared memory
    int kernelSize = 3;
    int numBlocks = height - kernelSize + 1;
    int threadsPerBlock = width - kernelSize + 1;
    std::cout << "numBlocks: " << numBlocks << std::endl;
    std::cout << "threadsPerBlock: " << threadsPerBlock << std::endl;

    // create cuda event for timing
    cudaEvent_t start, stop;
    cudaEventCreate(&start);
    cudaEventCreate(&stop);

    // start timing
    cudaEventRecord(start);
    convolutionKernelShared<<<numBlocks, threadsPerBlock, kernelSize * kernelSize * sizeof(float)>>>(d_input, d_output, width, height, d_kernel, kernelSize);
    cudaEventRecord(stop);

    // stop timing
    cudaEventSynchronize(stop);
    milliseconds = 0.0;
    cudaEventElapsedTime(&milliseconds, start, stop);
    std::cout << "Time taken: " << milliseconds << " ms" << std::endl;

    // copy the result back to the host
    cudaMemcpy(output, d_output, width * height * sizeof(float), cudaMemcpyDeviceToHost);

    // free device memory
    cudaFree(d_input);
    cudaFree(d_output);
    cudaFree(d_kernel);
}

int main(int argc, char *argv[]) {

    // ./program <input_image> <output_image> | ./program <input_image> <output_image> <-test>
    if (argc != 3 && argc != 4) {
        std::cerr << "Usage: " << argv[0] << " input_image output_image" << std::endl;
        return 1;
    }

    // Load input image
    int width, height, channels;
    unsigned char *inputImage = stbi_load(argv[1], &width, &height, &channels, 1);
    float *outputImageFloat = new float[width * height];

    if (!inputImage) {
        std::cerr << "Error loading image: " << stbi_failure_reason() << std::endl;
        return 1;
    }

    // print width and height
    std::cout << "width: " << width << std::endl;
    std::cout << "height: " << height << std::endl;

    // convert input to float
    float *inputImageFloat = new float[width * height];
    for (int i = 0; i < width * height; ++i) {
        inputImageFloat[i] = inputImage[i] / 255.0f;
    }

    float milliseconds = 0.0;
    float sum = 0.0;

    for (int i = 0; i < 10; ++i) {
        // apply Laplacian of Gaussian (LoG) filter using CUDA with shared memory
        applyLoGFilterCUDA(inputImageFloat, outputImageFloat, width, height, milliseconds);
        sum += milliseconds;
    }

    std::cout << "Average time taken: " << sum / 10.0 << " ms" << std::endl;

    // create output image buffer
    unsigned char *outputImage = new unsigned char[width * height];

    // convert output to unsigned char
    for (int i = 0; i < width * height; ++i) {
        outputImage[i] = (unsigned char) std::round(outputImageFloat[i] * 255.0f);
    }

    // save the output image
    stbi_write_png(argv[2], width, height, 1, outputImage, 0);

    // clean up
    stbi_image_free(inputImage);
    delete[] outputImage;
    delete[] outputImageFloat;

    std::cout << "Laplacian of Gaussian (LoG) edge detection completed successfully." << std::endl;

    return 0;
}


'File written in /content/src/conv_without_sh.cu'

In [18]:
!nvcc -arch=sm_75 -o "/content/src/conv_without_sh.o" /content/src/conv_without_sh.cu

/content/src/stb_image.h(4275): warning #550-D: variable "old_limit" was set but never used
     unsigned int cur, limit, old_limit;
                              ^

Remark: The warnings can be suppressed with "-diag-suppress <warning-number>"

/content/src/stb_image.h(5182): warning #550-D: variable "idata_limit_old" was set but never used
                 stbi__uint32 idata_limit_old = idata_limit;
                              ^

/content/src/stb_image.h(6969): warning #550-D: variable "out_size" was set but never used
        int out_size = 0;
            ^

/content/src/stb_image.h(6970): warning #550-D: variable "delays_size" was set but never used
        int delays_size = 0;
            ^



In [19]:
!/content/src/conv_without_sh.o /content/src/image_03.png /content/src/conv_03_3.png

width: 933
height: 882
numBlocks: 880
threadsPerBlock: 931
Time taken: 0.21136 ms
numBlocks: 880
threadsPerBlock: 931
Time taken: 0.099744 ms
numBlocks: 880
threadsPerBlock: 931
Time taken: 0.099904 ms
numBlocks: 880
threadsPerBlock: 931
Time taken: 0.098304 ms
numBlocks: 880
threadsPerBlock: 931
Time taken: 0.097728 ms
numBlocks: 880
threadsPerBlock: 931
Time taken: 0.097984 ms
numBlocks: 880
threadsPerBlock: 931
Time taken: 0.098304 ms
numBlocks: 880
threadsPerBlock: 931
Time taken: 0.108448 ms
numBlocks: 880
threadsPerBlock: 931
Time taken: 0.098208 ms
numBlocks: 880
threadsPerBlock: 931
Time taken: 0.098304 ms
Average time taken: 0.110829 ms
Laplacian of Gaussian (LoG) edge detection completed successfully.
